In [48]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score

import time
import datetime

from sklearn import preprocessing
from sklearn import metrics

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import NearestNeighbors

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

from sklearn.cluster import AffinityPropagation
from itertools import cycle
from sklearn.neighbors import KNeighborsClassifier


In [53]:
df_cell_train = pd.read_csv('sample_data.csv', index_col = None)
df_cell_train.shape

(102974, 9)

In [38]:

# Add some columns to make calculations easier
#df_cell_train["hour"] = (df_cell_train["time"]//60)%24+1
#df_cell_train["week"] = (df_cell_train['time']//1440)%7+1
#df_cell_train["month"] = (df_cell_train['time']//43200)%12+1
#df_cell_train['year'] = (df_cell_train['time']//525600)+1

#df_cell_train["hour"] = (df_cell_train["time"]%(60*24))/60.
#df_cell_train["dayofweek"] = np.ceil((df_cell_train["time"]%(60*24*7))/(60.*24))
#df_cell_train['dayofmonth'] = np.ceil((df_cell_train['time']%(60*24*30))/(60.*24))
#df_cell_train["dayofyear"] = np.ceil((df_cell_train["time"]%(60*24*365))/(60.*24))
#df_cell_train.head()


df_cell_train['x_y_mean'] = (df_cell_train['x']+df_cell_train['y'])/2
df_cell_train['x/y'] = (df_cell_train['x']/(df_cell_train['y']+0.01**100))

#df_cell_train = df_cell_train.drop(['row_id', 'time'], axis = 1)

In [39]:
#df_cell_train = df_cell_train.reset_index().drop('index', axis = 1)
df_cell_train.head()

,x,y,accuracy,place_id,month,hour,dayofweek,dayofmonth,dayofyear,x_y_mean,x/y
0,0.9536,0.3766,95,7578348615,4,15.633333,6,4,89,0.66510,2.532130
1,0.6733,0.8339,82,1273954023,4,11.366667,6,18,103,0.75360,0.807411
2,0.8700,0.0681,65,7131826933,1,8.133333,2,20,15,0.46905,12.775330
3,0.5939,0.0060,66,1809053339,7,4.333333,6,22,202,0.29995,98.983333
4,0.9637,0.2188,4,2198399075,7,8.416667,7,6,181,0.59125,4.404479


In [6]:
#df_cell_train.to_csv('sample_data.csv')

In [75]:
X = df_cell_train.drop('place_id', axis = 1)

y = pd.DataFrame(df_cell_train.place_id)

In [76]:
X.head()

,x,y,accuracy,month,hour,dayofweek,dayofmonth,dayofyear
0,0.9536,0.3766,95,4,15.633333,6,4,89
1,0.6733,0.8339,82,4,11.366667,6,18,103
2,0.8700,0.0681,65,1,8.133333,2,20,15
3,0.5939,0.0060,66,7,4.333333,6,22,202
4,0.9637,0.2188,4,7,8.416667,7,6,181


In [42]:
##########  normalized  ########

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
df_normalized = pd.DataFrame(np_scaled)
df_normalized.columns = X.columns
X_normalized = df_normalized

# Feature Engineering

In [77]:
#############   create range for accuracy   ##########

df_accuracy = X.accuracy

X['range_accuracy'] = X['accuracy'].apply(lambda x: 0 if x < 40 else (1 if x >= 40 and x < 80 else 2))
X.head()

,x,y,accuracy,month,hour,dayofweek,dayofmonth,dayofyear,range_accuracy
0,0.9536,0.3766,95,4,15.633333,6,4,89,2
1,0.6733,0.8339,82,4,11.366667,6,18,103,2
2,0.8700,0.0681,65,1,8.133333,2,20,15,1
3,0.5939,0.0060,66,7,4.333333,6,22,202,1
4,0.9637,0.2188,4,7,8.416667,7,6,181,0


In [63]:
# train, test split 

#X_new = X_new.drop(['range_accuracy', 'dayofyear', 'dayofweek', 'dayofmonth'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(82379, 10) (82379, 1)
(20595, 10) (20595, 1)


In [27]:
#  use k means cluster data points as a new feature 

kmeans = KMeans(n_clusters = 100)
location = X[['x', 'y']]
kmeans = kmeans.fit(location)
labels = kmeans.predict(location) 

In [28]:
labels = pd.DataFrame(labels,columns=['cluster_kmeans'])

In [78]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
df_normalized = pd.DataFrame(np_scaled)
df_normalized.columns = X.columns
X_normalized = df_normalized

In [79]:
X_normalized.head()

,x,y,accuracy,month,hour,dayofweek,dayofmonth,dayofyear,range_accuracy
0,0.907381,0.376638,0.092794,0.272727,0.651842,0.857143,0.133333,0.241758,1.0
1,0.346669,0.833983,0.079961,0.272727,0.473940,0.857143,0.600000,0.280220,1.0
2,0.740148,0.068107,0.063179,0.000000,0.339124,0.285714,0.666667,0.038462,0.5
3,0.187838,0.006001,0.064166,0.545455,0.180681,0.857143,0.733333,0.552198,0.5
4,0.927586,0.218822,0.002962,0.545455,0.350938,1.000000,0.200000,0.494505,0.0


In [34]:
labels = pd.DataFrame(labels,columns=['cluster_kmeans'])

X_new = pd.concat([X_normalized, labels], axis = 1)

X_new.head()

,x,y,accuracy,hour,dayofweek,dayofmonth,dayofyear,range_accuracy,cluster_kmeans
0,0.017902,0.232123,0.065153,0.349548,0.857143,0.566667,0.030220,0.5,42
1,0.148915,0.258626,0.057256,0.475330,0.428571,0.600000,0.293956,0.5,8
2,0.365937,0.153715,0.172754,0.247394,0.285714,0.366667,0.521978,1.0,80
3,0.953695,0.376638,0.092794,0.651842,0.857143,0.133333,0.241758,1.0,3
4,0.673367,0.833983,0.079961,0.473940,0.857143,0.600000,0.280220,1.0,83


In [43]:
#### train, test split 

#X_new = X_new.drop(['range_accuracy', 'dayofyear', 'dayofweek', 'dayofmonth'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size = 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(82379, 10) (82379, 1)
(20595, 10) (20595, 1)


# KNN

In [278]:
from sklearn.neighbors import KNeighborsClassifier   # n_neighbors = 10

classifier = KNeighborsClassifier(n_neighbors = 10 , weights = 'distance', metric = 'manhattan', n_jobs = 1, p = 2)
classifier.fit(X_train, y_train)


/Users/aoxu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='distance')

In [279]:
y_pred_knn = classifier.predict(X_test)
print('accuracy:', metrics.accuracy_score(y_test, y_pred_knn))

accuracy: 0.5430369127516779


# Decision Tree

In [283]:
tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 100, max_depth = 20, min_samples_leaf = 5)

tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

print('accuracy:', metrics.accuracy_score(y_test, y_pred_tree))

accuracy: 0.5176174496644296


# Random Forest

In [44]:
rf = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42, max_depth = 20)
rf.fit(X_train, y_train)

/Users/aoxu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [45]:
y_pred_rf = rf.predict(X_test)

print('accuracy:', metrics.accuracy_score(y_test, y_pred_rf))

accuracy: 0.7257101238164603


# Naive Bayes

In [46]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

print('accuracy:', metrics.accuracy_score(y_test, y_pred_nb))

/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.6117989803350328


# Neural Network

In [54]:
# perception 
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver = 'lbfgs', hidden_layer_sizes=(50,10), activation = 'logistic')
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

print('accuracy:', metrics.accuracy_score(y_test, y_pred_mlp))

/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.015149308084486526


# Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

rf = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 42, max_depth = 20)
adaboost = AdaBoostClassifier(base_estimator = rf, n_estimators = 20, learning_rate = 1)
adaboost.fit(X_train, y_train)
y_pred_ada = adaboost.predict(X_test) 

print('accuracy:', metrics.accuracy_score(y_test, y_pred_ada))

/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# XGBoost

In [103]:
import xgboost as xgb

xg_class = xgb.XGBClassifier(colsample_bytree=0.3, learning_rate=0.2, max_depth=20, alpha = 10, n_estimators= 10)
xg_class.fit(X_train, y_train)

y_pred_xgb = xg_class.predict(X_test)
print('accuracy:', metrics.accuracy_score(y_test, y_pred_xgb))

/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy: 0.9978506949419688


/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Stacking

In [11]:
###########  stacking  #######

from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors = 100 , weights = 'distance', metric = 'manhattan', n_jobs = 1, p = 2)

model1 = DecisionTreeClassifier(criterion = 'entropy', random_state = 100, max_depth = 20, min_samples_leaf = 5)
model2 = GaussianNB()
#model3 = KNeighborsClassifier(n_neighbors = 10 , weights = 'distance', metric = 'manhattan', n_jobs = 1, p = 2)

rf = RandomForestClassifier(n_estimators = 65, criterion = 'entropy', random_state = 42, max_depth = 20)

sclf = StackingClassifier(classifiers=[model1, model2], meta_classifier = rf, use_probas=True)

print('3-fold cross validation:')
for model, label in zip([model1, model2, sclf], ['dtree', 'nb', 'stacking']):
    scores = model_selection.cross_val_score(model, X, y, cv = 3, scoring = 'accuracy')
    print(scores)

3-fold cross validation:
[0.61257153 0.61200594 0.61305314]


/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.60673309 0.60466606 0.60714912]


/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aoxu/anaconda/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:171: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.meta_clf_.fit(meta_features, y)
/Users/aoxu/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aoxu/anaconda/lib/python3.6/site-packages/mlxtend/classifier/stacking_classification.py:171: DataConversionWarning: A column-vector y was passed when a 1d ar

[0.67107212 0.66766668 0.67177179]


In [84]:
len(X[0])

9

In [89]:
############    Neural Network   #########

num_inputs = 9
hidden_layer_neurons = 5   # set hidden layer number is 5

np.random.seed(4)
w1 = 2*np.random.random((num_inputs, hidden_layer_neurons)) - 1

num_outputs = 1
w2 = 2*np.random.random((hidden_layer_neurons, num_outputs)) - 1

In [90]:
from sklearn.model_selection import KFold

In [93]:
y = y.values

[[7578348615]
 [1273954023]
 [7131826933]
 ...
 [2411005523]
 [7809444487]
 [7269384886]]
[[7578348615]
 [1273954023]
 [7131826933]
 ...
 [1467576612]
 [7809444487]
 [7269384886]]
[[1273954023]
 [7131826933]
 [5861856288]
 ...
 [8353032062]
 [1467576612]
 [7269384886]]
[[7578348615]
 [1809053339]
 [2198399075]
 ...
 [2411005523]
 [1467576612]
 [7809444487]]
[[7578348615]
 [1273954023]
 [7131826933]
 ...
 [1467576612]
 [7809444487]
 [7269384886]]


In [96]:
folds = KFold(n_splits= 3, shuffle=True, random_state=100)


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, y)):

    print("Fold: ",n_fold+1)
    learning_rate = 0.2       # slowly update the network
    for epoch in range(1000):
        l1 = 1/(1 + np.exp(-(np.dot(X[train_idx], w1))))    # sigmoid function
        l2 = 1/(1 + np.exp(-(np.dot(l1, w2))))      # sigmoid function
        er = (abs(y[train_idx] - l2)).mean()       # error function
        l2_delta = (y[train_idx] - l2)*(l2 * (1-l2))
        l1_delta = l2_delta.dot(w2.T) * (l1 * (1-l1))
        w2 += l1.T.dot(l2_delta) * learning_rate        # gradient
        w1 += X[train_idx].T.dot(l1_delta) * learning_rate     # gradient
    print('Training Accuracy:', 1 - er)
        
        
    l1 = 1/(1 + np.exp(-(np.dot(X[valid_idx], w1))))
    l2 = 1/(1 + np.exp(-(np.dot(l1, w2))))
    yp = np.argmax(l2,axis=1)
    res = yp == np.argmax(y[valid_idx],axis=1)
    correct = np.sum(res) / len(res)
    print("Testing Accuracy: ",correct)

Fold:  1


/Users/aoxu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


Training Accuracy: -5489257396.02102
Testing Accuracy:  1.0
Fold:  2


/Users/aoxu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


Training Accuracy: -5508109481.978688
Testing Accuracy:  1.0
Fold:  3
Training Accuracy: -5519078838.919097
Testing Accuracy:  1.0
